In [2]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers import BatchNormalization
import numpy as np
import pylab as plt

In [4]:
# filters: 卷积核数量
# kernel_size: 卷积核大小
# strides: 卷积核往右和往下移动的步长
# padding: 处理边界的策略，valid表示不处理边界，输出shape会变小；same表示处理边界，输出shape和输入shape一致
# return_sequences: 是否返回中间序列，true表示输出所有输出，false表示只输出最后一个时间的输出
seq = Sequential()
seq.add(ConvLSTM2D(filters=16, kernel_size=(3, 3), strides=(1,1),
                   input_shape=(None, 16, 16, 1),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())
seq.add(Dropout(0.3))

seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())
seq.add(Dropout(0.3))

seq.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())
seq.add(Dropout(0.5))


seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.compile(loss='binary_crossentropy', optimizer='nadam')
seq.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_3 (ConvLSTM2D)  (None, None, 16, 16, 16)  9856      
                                                                 
 batch_normalization_3 (Batc  (None, None, 16, 16, 16)  64       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, None, 16, 16, 16)  0         
                                                                 
 conv_lstm2d_4 (ConvLSTM2D)  (None, None, 16, 16, 32)  55424     
                                                                 
 batch_normalization_4 (Batc  (None, None, 16, 16, 32)  128      
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, None, 16, 16, 32) 

In [8]:
def generate_movies(n_samples=1200, n_frames=15):
    row = 32
    col = 32
    shifted_movies = np.zeros((n_samples, n_frames, row, col, 1),
                              dtype=np.float)

    for i in range(n_samples):
        # Add 3 to 7 moving squares
        n = np.random.randint(3, 8)

        for j in range(n):
            # Initial position
            xstart = np.random.randint(8, 24)
            ystart = np.random.randint(8, 24)
            # Direction of motion
            directionx = np.random.randint(0, 3) - 1
            directiony = np.random.randint(0, 3) - 1

            # Size of the square
            w = np.random.randint(2, 4)

            for t in range(n_frames):
                x_shift = xstart + directionx * t
                y_shift = ystart + directiony * t

                x_shift = xstart + directionx * (t + 1)
                y_shift = ystart + directiony * (t + 1)
                shifted_movies[i, t, x_shift - w: x_shift + w,
                               y_shift - w: y_shift + w, 0] += 1

    shifted_movies = shifted_movies[::, ::, 8:24, 8:24, ::]
    shifted_movies[shifted_movies >= 1] = 1
    return shifted_movies

shifted_movies = generate_movies(n_samples=1200)
shifted_movies.shape

D:\ZJU\Anaconda3\envs\learned_index\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """


(1200, 15, 16, 16, 1)

In [10]:
# x为前1000个数据中每个数据的0-10个图片
# y为前1000个数据中每个数据的5-15个图片，因此预测5次，步长为10
# validation_split: 用于做验证集的数据百分比
seq.fit(shifted_movies[:1000, 0:10], shifted_movies[:1000, 5:15], batch_size=10,
        epochs=20, validation_split=0.05)

Epoch 1/20
95/95 [==============================] - 25s 134ms/step - loss: 0.4595 - val_loss: 0.5111
Epoch 2/20
95/95 [==============================] - 12s 131ms/step - loss: 0.2866 - val_loss: 0.4138
Epoch 3/20
95/95 [==============================] - 12s 130ms/step - loss: 0.2438 - val_loss: 0.3649
Epoch 4/20
95/95 [==============================] - 12s 127ms/step - loss: 0.2179 - val_loss: 0.3286
Epoch 5/20
95/95 [==============================] - 12s 129ms/step - loss: 0.2013 - val_loss: 0.2628
Epoch 6/20
95/95 [==============================] - 12s 128ms/step - loss: 0.1883 - val_loss: 0.1754
Epoch 7/20
95/95 [==============================] - 13s 136ms/step - loss: 0.1799 - val_loss: 0.1626
Epoch 8/20
95/95 [==============================] - 13s 140ms/step - loss: 0.1714 - val_loss: 0.1550
Epoch 9/20
95/95 [==============================] - 13s 139ms/step - loss: 0.1661 - val_loss: 0.1498
Epoch 10/20
95/95 [==============================] - 13s 135ms/step - loss: 0.1606 - val_lo

In [13]:
x_true = shifted_movies[:1000, 0:10]
x_true[1].shape

(10, 16, 16, 1)

In [11]:
x_true = shifted_movies[:1000, 0:10]
y_true =
y_pred = seq.predict(shifted_movies[1000:, 0:10])